In [2]:
import os
import pickle
import pandas as pd
import numpy as np

##### Path to outputdict file

In [13]:
output_dict_path = "outputDictNew" #path to output dict

In [14]:
os.path.isfile(output_dict_path)

True

##### Load the file

In [15]:
output_dict = pickle.load(open(output_dict_path,"rb"))

In [16]:
output_dict

,Labels,Prob,Logits,ethnicity,gender,age
0,0.0,0.005472,-5.202624,BLACK/AFRICAN AMERICAN,F,57
1,0.0,0.001005,-6.902061,BLACK/AFRICAN AMERICAN,M,68
2,0.0,0.019099,-3.938828,WHITE,F,87
3,0.0,0.008335,-4.778972,WHITE,M,46
4,0.0,0.004045,-5.506170,WHITE,M,68
...,...,...,...,...,...,...
7895,0.0,0.003010,-5.802665,BLACK/AFRICAN AMERICAN,F,31
7896,0.0,0.033274,-3.369145,WHITE,M,83
7897,0.0,0.013597,-4.284217,HISPANIC/LATINO,M,35
7898,1.0,0.305867,-0.819511,WHITE,M,75


##### Add predicted label using the probability

In [19]:
output_dict["Predicted"] = output_dict.apply(lambda row:1 if row.Prob>=0.5 else 0, axis=1)

In [20]:
output_dict

,Labels,Prob,Logits,ethnicity,gender,age,Predicted
0,0.0,0.005472,-5.202624,BLACK/AFRICAN AMERICAN,F,57,0
1,0.0,0.001005,-6.902061,BLACK/AFRICAN AMERICAN,M,68,0
2,0.0,0.019099,-3.938828,WHITE,F,87,0
3,0.0,0.008335,-4.778972,WHITE,M,46,0
4,0.0,0.004045,-5.506170,WHITE,M,68,0
...,...,...,...,...,...,...,...
7895,0.0,0.003010,-5.802665,BLACK/AFRICAN AMERICAN,F,31,0
7896,0.0,0.033274,-3.369145,WHITE,M,83,0
7897,0.0,0.013597,-4.284217,HISPANIC/LATINO,M,35,0
7898,1.0,0.305867,-0.819511,WHITE,M,75,0


##### Bin age

In [21]:
output_dict["age_binned"] = output_dict.age.apply(lambda x:"{}-{}".format((x//10)*10,(x//10 + 1)*10))

##### List of sensitive columns

In [22]:
sensitive_columns = ["ethnicity", "gender", "age_binned"]

##### Function to compute the Confusion Matrix parameters

In [29]:
def get_cm_parameters(gt, pred):
    zipped_gt_pred = list(zip(gt,pred))
    tp = len([pair for pair in zipped_gt_pred if pair == (1,1)])
    tn = len([pair for pair in zipped_gt_pred if pair == (0,0)])
    fp = len([pair for pair in zipped_gt_pred if pair == (0,1)])
    fn = len([pair for pair in zipped_gt_pred if pair == (1,0)])
    
    try:
        tpr = tp/(tp + fn)
    except ZeroDivisionError:
        tpr = None
    try:
        tnr = tn/(tn + fp)
    except ZeroDivisionError:
        tnr = None
    try:
        fpr = fp/(fp + tn)
    except ZeroDivisionError:
        fpr = None
    try:
        fnr = fn/(fn + tp)
    except ZeroDivisionError:
        fnr = None
    try:
        pr = (tp + fp)/(len(zipped_gt_pred))
    except:
        pr = None
    try:
        nr = (tn + fn)/(len(zipped_gt_pred))
    except:
        nr = None
    try:
        acc = (tp+tn)/(len(zipped_gt_pred))
    except ZeroDivisionError:
        acc = None
    
    return tp, tn, fp, fn, tpr, tnr, fpr, fnr, pr, nr, acc

##### Generate fairness report

In [30]:
report_list = []
for sens_col in sensitive_columns:
    for group, aggregate in output_dict.groupby(sens_col):
        tmp_dct = {"sensitive_attribute": sens_col}
        tp, tn, fp, fn, tpr, tnr, fpr, fnr, pr, nr, acc = get_cm_parameters(list(aggregate.Labels), list(aggregate.Predicted))
        tmp_dct.update(dict(
            group=group,tp=tp, tn=tn, fp=fp, fn=fn, tpr=tpr, tnr=tnr, fpr= fpr, fnr=fnr, pr=pr, nr=nr, accuracy=acc    
            )
        )
        report_list.append(tmp_dct)

In [31]:
report = pd.DataFrame(report_list)
report_groups = {c:i for i,c in enumerate(report.sensitive_attribute.unique())}

In [32]:
def highlight(s):
    colors = [['background-color: yellow'], ['background-color: green'], ['background-color: red']]
    return colors[report_groups[s.sensitive_attribute]%len(colors)] * len(s)

In [33]:
try:
    import jinja2
    display(report.style.apply(highlight, axis=1))
except ImportError:
    display(report)

,sensitive_attribute,group,tp,tn,fp,fn,tpr,tnr,fpr,fnr,pr,nr,accuracy
0,ethnicity,AMERICAN INDIAN/ALASKA NATIVE,0,24,0,0,nan,1.000000,0.000000,nan,0.000000,1.000000,1.000000
1,ethnicity,ASIAN,0,194,1,13,0.000000,0.994872,0.005128,1.000000,0.004808,0.995192,0.932692
2,ethnicity,BLACK/AFRICAN AMERICAN,0,1632,4,47,0.000000,0.997555,0.002445,1.000000,0.002377,0.997623,0.969697
3,ethnicity,HISPANIC/LATINO,2,375,1,12,0.142857,0.997340,0.002660,0.857143,0.007692,0.992308,0.966667
4,ethnicity,OTHER,1,293,0,10,0.090909,1.000000,0.000000,0.909091,0.003289,0.996711,0.967105
5,ethnicity,UNABLE TO OBTAIN,1,16,0,0,1.000000,1.000000,0.000000,0.000000,0.058824,0.941176,1.000000
6,ethnicity,UNKNOWN,4,138,5,23,0.148148,0.965035,0.034965,0.851852,0.052941,0.947059,0.835294
7,ethnicity,WHITE,17,4834,35,218,0.072340,0.992812,0.007188,0.927660,0.010188,0.989812,0.950431
8,gender,F,7,3159,18,115,0.057377,0.994334,0.005666,0.942623,0.007578,0.992422,0.959685
9,gender,M,18,4347,28,208,0.079646,0.993600,0.006400,0.920354,0.009998,0.990002,0.948707
